In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


import numpy as np
import pandas as pd
import warnings
import sweetviz as sv


warnings.filterwarnings("ignore")





Using matplotlib backend: MacOSX


In [89]:
df = pd.read_csv('sales.csv')
test = pd.read_csv('real_data.csv')


In [67]:
df.head(5)

,store_ID,day_of_week,nb_customers_on_day,open,promotion,state_holiday,school_holiday,sales,year,month,day
0,166,2,0,0,0,1,1,0,2013,1,1
1,278,2,0,0,0,1,1,0,2013,1,1
2,519,2,0,0,0,1,1,0,2013,1,1
3,234,2,0,0,0,1,1,0,2013,1,1
4,270,2,0,0,0,1,1,0,2013,1,1


In [14]:
test.head(5)

,Unnamed: 0,store_ID,day_of_week,date,nb_customers_on_day,open,promotion,state_holiday,school_holiday
0,419571,637,7,2015-03-01,0,0,0,0,0
1,378043,801,7,2015-03-01,0,0,0,0,0
2,282522,354,7,2015-03-01,0,0,0,0,0
3,77715,371,7,2015-03-01,0,0,0,0,0
4,205875,515,7,2015-03-01,0,0,0,0,0


In [90]:
df.drop('Unnamed: 0' ,axis=1,inplace=True)
test.drop('Unnamed: 0' ,axis=1,inplace=True)

In [91]:
df['date'] =pd.to_datetime(df['date'])
test['date'] =pd.to_datetime(test['date'])



In [ ]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

df.drop(columns=['date'], inplace=True)

In [ ]:
test['year'] = test['date'].dt.year
test['month'] = test['date'].dt.month
test['day'] = test['date'].dt.day

test.drop(columns=['date'], inplace=True)

In [92]:
df['state_holiday'].replace({'a': 1, 'b': 2 ,'c' :3},inplace=True)
test['state_holiday'].replace({'a': 1, 'b': 2 ,'c' :3},inplace=True)



In [93]:
df['state_holiday'] =df['state_holiday'].astype(int)
test['state_holiday'] =test['state_holiday'].astype(int)

In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 592561 entries, 0 to 592560
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   store_ID             592561 non-null  int64         
 1   day_of_week          592561 non-null  int64         
 2   date                 592561 non-null  datetime64[ns]
 3   nb_customers_on_day  592561 non-null  int64         
 4   open                 592561 non-null  int64         
 5   promotion            592561 non-null  int64         
 6   state_holiday        592561 non-null  int64         
 7   school_holiday       592561 non-null  int64         
 8   sales                592561 non-null  int64         
dtypes: datetime64[ns](1), int64(8)
memory usage: 40.7 MB


# The best library in visualisation  

In [ ]:
report = sv.analyze(df)

# Display the report
report.show_html('data_report.html')

                                             |          | [  0%]   00:00 -> (? left)

Report data_report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [ ]:
X = df.drop('sales',axis=1) 
y = df['sales']

X_train , X_test , y_train , y_test =train_test_split(X,y,
                                                    shuffle=False,
                                                     test_size=0.5
                           
                                              )

# 50% Trainig 
# 50% Testing

In [ ]:
model = GradientBoostingRegressor(
    n_estimators=500,     
    max_depth=6,         
    learning_rate=0.1,    
    subsample=0.8          
)


# We tried about 10 models but the best R**2 is GradientBoostingRegressor

In [212]:
model.fit(X_train,y_train)


GradientBoostingRegressor(max_depth=6, n_estimators=500, subsample=0.8)

In [217]:

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)


r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)
mae = mean_absolute_error(y_test, y_test_pred)
mse = mean_squared_error(y_test, y_test_pred)
rmse = np.sqrt(mse)


print(f"\nR2 score on train set: {r2_train:.2}")
print(f"R2 score on test set: {r2_test:.2}\n")
print(f"Mean Absolute Error (MAE): {mae:.5}\n")
print(f"Mean Squared Error (MSE): {mse:.7}\n")
print(f"Root Mean Squared Error (RMSE): {rmse:.5}\n")







R2 score on train set: 0.98
R2 score on test set: 0.97

Mean Absolute Error (MAE): 468.24

Mean Squared Error (MSE): 465150.2

Root Mean Squared Error (RMSE): 682.02



# R2 = 97% 

# We are now predicting the test file.

In [218]:
model.fit(X,y)

GradientBoostingRegressor(max_depth=6, n_estimators=500, subsample=0.8)

In [219]:
y_pred_real_file = model.predict(test)


In [241]:
test_not_clean = pd.read_csv('real_data.csv')
test_not_clean['sales']=y_pred_real_file
test_not_clean.to_csv('pred_real_data.csv')

### solving the sales  0 

# <3

In [242]:
df_pred = pd.read_csv('pred_real_data.csv')
df_pred.loc[df_pred['open'] == 0, 'sales'] = 0
df_pred['sales'].astype(int)
df_pred['sales']=df_pred['sales'].astype(int)
df_pred.to_csv('pred_real_data.csv')